# Historical Data Inference

In [1]:
import sys
sys.path.append('/Users/captainrib/workspace/project-ares/ares-data-processing/aresdataprocessing')

from roboflow import Roboflow
import json, os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from processor.segementation_generation import SegGen
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import matplotlib.dates as mdates
import plotly.graph_objects as go
import numpy as np
import math

In [2]:
rf = Roboflow(api_key="rjip0pPqsC01gzM4itzX")
project = rf.workspace().project("project-ares")
model = project.version(1).model
#image_path = '/Users/captainrib/workspace/project-ares/ares-data-processing/data/images/testcases/pos2.png'

def save_image(image_path, detections, output_path):
    image = Image.open(image_path)

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for box in detections:
        color = "#4892EA"
        x1 = box['x'] - box['width'] / 2
        x2 = box['x'] + box['width'] / 2
        y1 = box['y'] - box['height'] / 2
        y2 = box['y'] + box['height'] / 2

        draw.rectangle([
            x1, y1, x2, y2
        ], outline=color, width=1)

    image.save(output_path)

loading Roboflow workspace...
loading Roboflow project...


In [3]:
input_folder = '/Users/captainrib/workspace/project-ares/ares-data-processing/data/images/newdata'
output_folder = '/Users/captainrib/workspace/project-ares/ares-data-processing/data/images/haspattern'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, filename)

        # Get predictions using your model (replace this line with actual model predictions)
        result = model.predict(input_image_path, confidence=70, overlap=30).json()
        detections = result['predictions']
        
        if detections:
            save_image(input_image_path, detections, output_image_path)

In [4]:
data_path = '/Users/captainrib/workspace/project-ares/ares-data-processing/data/raw/'
date = '2023-03-23'
data = pd.read_csv('{}amd_{}.csv'.format(data_path, date), parse_dates=['timestamp'])
segment_size = 30
start = 85

segment_data = data.iloc[start:start + segment_size].reset_index()
segment_data['timestamp'] = mdates.date2num(segment_data['timestamp'].dt.to_pydatetime())
print(segment_data['timestamp'])
fig = go.Figure(data=[go.Candlestick(x=segment_data['timestamp'],
                open=segment_data['open'],
                high=segment_data['high'],
                low=segment_data['low'],
                close=segment_data['close'], increasing_line_color='red', decreasing_line_color='red')])

# Remove slider, set background color to white and remove grid
fig.update_layout(xaxis_rangeslider_visible=False, paper_bgcolor='white', xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
fig.update_traces(line=dict(width=0), showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.write_image('example.png')

0     19439.621528
1     19439.622222
2     19439.622917
3     19439.623611
4     19439.624306
5     19439.625000
6     19439.625694
7     19439.626389
8     19439.627083
9     19439.627778
10    19439.628472
11    19439.629167
12    19439.629861
13    19439.630556
14    19439.631250
15    19439.631944
16    19439.632639
17    19439.633333
18    19439.634028
19    19439.634722
20    19439.635417
21    19439.636111
22    19439.636806
23    19439.637500
24    19439.638194
25    19439.638889
26    19439.639583
27    19439.640278
28    19439.640972
29    19439.641667
Name: timestamp, dtype: float64


In [5]:
result = model.predict('example.png', confidence=70, overlap=30).json()
detections = result['predictions']
print(detections)
save_image('example.png', detections, 'example_i.png')

[{'x': 266, 'y': 255, 'width': 86, 'height': 156, 'confidence': 0.7688460350036621, 'class': 'ABCD', 'image_path': 'example.png', 'prediction_type': 'ObjectDetectionModel'}]


In [6]:

def get_timestamps(segement_time, start_timestamp, end_timestamp, image_width, box_x, box_width):
    start_index = int(box_x - box_width / 2 - 80)
    end_index = int(start_index + box_width)
    start_timestamp = math.ceil(start_index) // 18
    end_timestamp = math.ceil(end_index) // 18
    start_pattern_timestamp = segement_time[start_timestamp]
    end_pattern_timestamp = segement_time[end_timestamp]

    return start_pattern_timestamp, end_pattern_timestamp

image_width = 700
start_timestamp = segment_data['timestamp'][0]
end_timestamp = segment_data['timestamp'][29]
box_x = detections[0]['x']
box_width = detections[0]['width']

print(segment_data['timestamp'])
start_pattern_timestamp, end_pattern_timestamp = get_timestamps(segment_data['timestamp'],
    start_timestamp, end_timestamp, image_width, box_x, box_width
)

0     19439.621528
1     19439.622222
2     19439.622917
3     19439.623611
4     19439.624306
5     19439.625000
6     19439.625694
7     19439.626389
8     19439.627083
9     19439.627778
10    19439.628472
11    19439.629167
12    19439.629861
13    19439.630556
14    19439.631250
15    19439.631944
16    19439.632639
17    19439.633333
18    19439.634028
19    19439.634722
20    19439.635417
21    19439.636111
22    19439.636806
23    19439.637500
24    19439.638194
25    19439.638889
26    19439.639583
27    19439.640278
28    19439.640972
29    19439.641667
Name: timestamp, dtype: float64


In [7]:
print(start_pattern_timestamp)
print(end_pattern_timestamp)

19439.62638888889
19439.629861111112
